In [ ]:
!pip install chess

In [ ]:
import chess
import chess.pgn

In [ ]:
startMoves = "d4 d5 e4 dxe4 Nc3 Nf6 f3 exf3 Nxf3".split(" ")
pgnFile = "./bdg-games-tiny.pgn"
# pgnFile = "./D00-bdg-games.pgn"

In [ ]:
board = chess.Board()
[board.push_san(move) for move in startMoves]

startFen = board.fen()
print(f"[FEN] {startFen}")
board

In [ ]:
def get_game_headers(game):
    return {
        'Id': f"chesscom:{game.headers['ChesscomId']}",
        'Site': game.headers["Site"],
        'Event': game.headers["Event"],
        'Date': game.headers["Date"],
        'Round': game.headers["Round"],
        'Result': game.headers["Result"],
        'White': game.headers["White"],
        'WhiteElo': game.headers["WhiteElo"],
        'Black': game.headers["Black"],
        'BlackElo': game.headers["BlackElo"],
        'ECO': game.headers["ECO"]
    }

def get_game_moves(game):
    return game.mainline_moves()

def get_simple_game(pgnGame):
    headers = get_game_headers(pgnGame)
    moves = get_game_moves(pgnGame)
    return {
        'headers': headers,
        'moves': moves
    }

def print_game_row(game, gameNo):
    white = f"{game['headers']['White']}({game['headers']['WhiteElo']})"
    black = f"{game['headers']['Black']}({game['headers']['BlackElo']})"
    players = f"{white} - {black}".ljust(56)[:56]
    date = game['headers']['Date']
    result = game['headers']['Result'][:3]
    print(f"[{gameNo.zfill(5)}] {players}{date} {result} ")

def position_fen(fen):
    return " ".join(fen.split(" ")[:4])

def dump_tree(tree):
    buffer = []

    for fromFen, node in tree.nodes.items():
        print(f"[FEN]  {fromFen}")
        for move, moveData in node.items():
            print(f"[MOVE] {move} {moveData['toFen']} {moveData['visits']}")


In [ ]:
class PosNode:
    def __init__(self, fen):
        self.moves = []
        self.gameRefs = []


class ChessTree:
    def __init__(self, startFen):
        self.startFen = position_fen(startFen)
        self.gameRefs = {}
        self.nodes = {}
        self.gameCount = 0

    def find_node(self, fromFen):
        return self.nodes.get(fromFen, {})

    def save_move(self, fromFen, move, toFen):
        node = self.find_node(fromFen)

        if move in node:
            node[move]['visits'] += 1
        else:
            node[move] = {
                'toFen': toFen,
                'visits': 1,
            }

        self.nodes[fromFen] = node

    def add_game(self, pgnGame, limitDepth=20):
        # print(f"[TREE] Adding game {self.gameCount + 1}")
        isStartPos = False
        board = pgnGame.board()
        prevFen = None
        depth = 0
        # gameRef = get_game_ref(pgnGame)

        for move in pgnGame.mainline_moves():
            board.push(move)
            boardFen = position_fen(board.fen())

            # Move through game until we reach the startFen position
            if isStartPos is False:
                # print(f"[SKIP] {move} {boardFen}")
                if boardFen == self.startFen:
                    isStartPos = True
                    prevFen = boardFen
                    # print("[TREE] Found start pos")
                    self.gameCount += 1
                continue

            # Add this game's position to the tree
            # print(f"[MOVE] {move} {boardFen}")
            node = self.save_move(prevFen, str(move), boardFen)
            depth += 1
            prevFen = boardFen

            if depth >= limitDepth:
                break



In [ ]:
pgn = open(pgnFile)
tree = ChessTree(startFen)
count = 0
lastDate = ''
maxGames = 10000

def track_progress():
    global count
    if count % 1000 == 0:
        print("#", end='')

while pgnGame := chess.pgn.read_game(pgn):
    # game = get_simple_game(pgnGame)
    tree.add_game(pgnGame)
    count += 1
    track_progress()
    # print_game_row(game, count)

    if count >= maxGames:
        break

print(f"[PGN] Imported {count} games.")
print()

dump_tree(tree)